# 配置环境

In [ ]:
!git clone https://github.com/Li-Kira/VITS_WebUI.git
%cd VITS_WebUI

Cloning into 'VITS_WebUI'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 39 (delta 4), reused 35 (delta 3), pack-reused 0
Unpacking objects: 100% (39/39), 39.94 KiB | 2.85 MiB/s, done.
/content/VITS_WebUI


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 50.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyopenjtalk: filename=pyopenjtalk-0.3.0-cp38-cp38-linux_x86_64.whl size=5865209 sha256=488be697a9d276d16a1dccdc24b2108e94ed30e4278ba5aec01dfa08f5af2ca0
  Stored in directory: /root/.cache/pip/wheels/88/74/65/3fffc9c6c6a4b625cd5108d4e42e599d6aee5fcf02b1480119
Successfully built pyopenjtalk


In [ ]:
%cd monotonic_align
!python setup.py build_ext --inplace
%cd ..

/content/VITS_WebUI/monotonic_align
Compiling core.pyx because it changed.
[1/1] Cythonizing core.pyx
/usr/local/lib/python3.8/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/VITS_WebUI/monotonic_align/core.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running build_ext
building 'monotonic_align.core' extension
creating build
creating build/temp.linux-x86_64-3.8
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.8/dist-packages/numpy/core/include -I/usr/include/python3.8 -c core.c -o build/temp.linux-x86_64-3.8/core.o
creating /content/VITS_WebUI/monotonic_align/monotonic_align
x86_64-linux-gnu-gcc -pthread -sha

In [ ]:
#@title 验证是否开启GPU
!nvidia-smi

Wed Mar  8 09:22:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0    23W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 数据集预处理

In [ ]:
#@title Download pre-trained model
!wget -nc -P ../drive/MyDrive/ML_Folder/Pytorch/Vits https://huggingface.co/spaces/sayashi/vits-uma-genshin-honkai/resolve/main/model/G_0.pth
!wget -nc -P ../drive/MyDrive/ML_Folder/Pytorch/Vits https://huggingface.co/spaces/sayashi/vits-uma-genshin-honkai/resolve/main/model/D_0.pth

In [ ]:
#@title clean txt
!python preprocess.py --filelists filelists/yuuka_train.txt filelists/yuuka_val.txt

START: filelists/yuuka_train.txt
Downloading: "https://github.com/r9y9/open_jtalk/releases/download/v1.11.1/open_jtalk_dic_utf_8-1.11.tar.gz"
dic.tar.gz: 100% 22.6M/22.6M [00:01<00:00, 20.2MB/s]
Extracting tar file /usr/local/lib/python3.8/dist-packages/pyopenjtalk/dic.tar.gz
START: filelists/yuuka_val.txt


In [ ]:
#@title  Edit config
import json
batchsize = 16  #@param {type:"number"}
training_files = "filelists/yuuka_train.txt.cleaned" #@param {type:"string"}
validation_files = "filelists/yuuka_val.txt.cleaned" #@param {type:"string"}
config = json.load(open("configs/config.json"))
config['train']['batch_size'] = batchsize
config['data']['training_files'] = training_files
config['data']['validation_files'] = validation_files
with open("configs/config.json", 'w+') as f:
    json.dump(config, f, indent=4)

# Training

In [ ]:
#@title Start Training
!python train_ms.py -c configs/config.json -m checkpoints

# 推理

In [ ]:
#@title  Infer
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch


import commons
import utils
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence
from scipy.io.wavfile import write


In [ ]:
def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

In [ ]:
config_path = "configs/config.json" #@param {type:"string"}
model_path = "../drive/MyDrive/vits-finetune/checkpoints/G_4000.pth" #@param {type:"string"}
hps = utils.get_hparams_from_file(config_path)
net_g = SynthesizerTrn(
    len(hps.symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).cuda()
model = net_g.eval()
model = utils.load_checkpoint(model_path, net_g, None)

In [ ]:
speaker_id = 0 #@param {type:"number"}
text = "\u306A\u3093\u3067\u7121\u69D8\u306A\u6700\u5F8C\u304B\u3057\u3089\u3002\u3042\u306A\u305F\u3001\u6B7B\u306B\u306A\u3063\u3066\u3044\u308B\u308F\u3088\u3002\u5206\u304B\u3063\u3066\u3044\u308B\u306E\uFF1F" #@param {type:"string"}
noise_scale=0.6 #@param {type:"number"}
noise_scale_w=0.668 #@param {type:"number"}
length_scale=1.0 #@param {type:"number"}
stn_tst = get_text(text, hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    sid = torch.LongTensor([speaker_id]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=noise_scale, noise_scale_w=noise_scale_w, length_scale=length_scale)[0][0,0].data.cpu().float().numpy()

ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))